In [91]:
# # Задание 1. (5 баллов) 
# # В тетрадке реализована биграмная языковая модель (при генерации учитывается информация только о 1
# предыдущем слове). Реализуйте триграмную модель и сгенерируйте несколько текстов. Сравните их с текстами,
# сгенерированными биграмной моделью. 
# # Можно использовать те же тексты, что в семинаре, или взять какой-то другой (на английском или русском языке).

In [92]:

dostoevsky = open('besy_dostoevsky.txt', encoding='cp1251').read()
tolstoy = open('anna_karenina.txt', encoding='utf-8').read()
dostoevsky = dostoevsky[:10000]
tolstoy = tolstoy[:10000]

In [93]:
from string import punctuation
import numpy as np

def normalize(text):
    normalized_text = [word.strip(punctuation) for word in text.lower().split()]
    normalized_text = [word for word in normalized_text if word]
    return normalized_text

In [94]:
norm_dostoevsky = normalize(dostoevsky)
norm_tolstoy = normalize(tolstoy)

In [95]:
print(norm_dostoevsky[:10])
print(len(norm_dostoevsky))
print(norm_tolstoy[:10])
print(len(norm_tolstoy))

['спасибо', 'что', 'скачали', 'книгу', 'в', 'бесплатной', 'электронной', 'библиотеке', 'royallib.ru', 'http://royallib.ru']
1507
['annotation', '«анна', 'каренина»', '–', 'это', 'сложное', 'психологически', 'утонченное', 'остропроблемное', 'произведение']
1587


In [96]:
print("Уникальных лемм в срезе Бесов -", len(set(norm_dostoevsky)))
print("Уникальный лемм в срезе Анны Карениной - ", len(set(norm_tolstoy)))

Уникальных лемм в срезе Бесов - 836
Уникальный лемм в срезе Анны Карениной -  817


In [97]:
from collections import Counter

In [98]:
vocab_dostoevsky = Counter(norm_dostoevsky)
vocab_tolstoy = Counter(norm_tolstoy)


In [99]:
vocab_dostoevsky.most_common(10)

[('и', 83),
 ('в', 57),
 ('что', 30),
 ('не', 28),
 ('его', 19),
 ('но', 19),
 ('он', 16),
 ('же', 15),
 ('с', 15),
 ('а', 11)]

In [100]:
vocab_tolstoy.most_common(10)

[('и', 70),
 ('–', 52),
 ('в', 47),
 ('не', 36),
 ('он', 36),
 ('что', 27),
 ('на', 25),
 ('с', 25),
 ('как', 22),
 ('матвей', 13)]

In [101]:
probas_dosoevsky = Counter({word:c/len(norm_dostoevsky) for word, c in vocab_dostoevsky.items()})
probas_dosoevsky.most_common(10)

[('и', 0.055076310550763105),
 ('в', 0.0378234903782349),
 ('что', 0.019907100199071003),
 ('не', 0.0185799601857996),
 ('его', 0.012607830126078301),
 ('но', 0.012607830126078301),
 ('он', 0.010617120106171201),
 ('же', 0.009953550099535502),
 ('с', 0.009953550099535502),
 ('а', 0.0072992700729927005)]

In [102]:
probas_tolstoy = Counter({word:c/len(norm_tolstoy) for word, c in vocab_tolstoy.items()})
probas_tolstoy.most_common(10)

[('и', 0.04410838059231254),
 ('–', 0.03276622558286074),
 ('в', 0.029615626969124134),
 ('не', 0.022684310018903593),
 ('он', 0.022684310018903593),
 ('что', 0.017013232514177693),
 ('на', 0.01575299306868305),
 ('с', 0.01575299306868305),
 ('как', 0.013862633900441084),
 ('матвей', 0.008191556395715185)]

In [103]:
phrase = 'Все смешалось в доме облонских'

prob = Counter({'tolstoy':0, 'dostoevsky':0})

for word in normalize(phrase):
    prob['dostoevsky'] += np.log(probas_dosoevsky.get(word, 0.00001))
    prob['tolstoy'] += np.log(probas_tolstoy.get(word, 0.00001))

prob.most_common()    

[('tolstoy', -28.392685814933895), ('dostoevsky', -43.52203961189503)]

In [104]:
from nltk.tokenize import sent_tokenize
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [105]:
sentences_dostoevsky = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dostoevsky)]
sentences_tolstoy = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(tolstoy)]

In [106]:
unigrams_dostoevsky = Counter()
bigrams_dostoevsky = Counter()
trigrams_dostoevsky = Counter()

for sentence in sentences_dostoevsky:
    unigrams_dostoevsky.update(sentence)
    bigrams_dostoevsky.update(ngrammer(sentence))
for sentence in sentences_dostoevsky:
    trigrams_dostoevsky.update(ngrammer(sentence, n=3))



unigrams_tolstoy = Counter()
bigrams_tolstoy = Counter()
trigrams_tolstoy = Counter()

for sentence in sentences_tolstoy:
    unigrams_tolstoy.update(sentence)
    bigrams_tolstoy.update(ngrammer(sentence))
for sentence in sentences_tolstoy:
    trigrams_tolstoy.update(ngrammer(sentence, n=3))

In [107]:
bi_matrix_dostoevsky = np.zeros((len(unigrams_dostoevsky), 
                   len(unigrams_dostoevsky)))
id2word_dostoevsky = list(unigrams_dostoevsky)
word2id_dostoevsky = {word:i for i, word in enumerate(id2word_dostoevsky)}


for ngram in bigrams_dostoevsky:
    word1, word2 = ngram.split()
    bi_matrix_dostoevsky[word2id_dostoevsky[word1]][word2id_dostoevsky[word2]] =  (bigrams_dostoevsky[ngram]/
                                                                     unigrams_dostoevsky[word1])

In [108]:
tri_matrix_dostoevsky = np.zeros((len(trigrams_dostoevsky), 
                   len(trigrams_dostoevsky)))
bi_id2word_dostoevsky = list(bigrams_dostoevsky)
bi_word2id_dostoevsky = {word:i for i, word in enumerate(bi_id2word_dostoevsky)}


for ngram in trigrams_dostoevsky:
    word1, word2, word3 = ngram.split()
    tri_matrix_dostoevsky[bi_word2id_dostoevsky[word1 + ' ' + word2]][word2id_dostoevsky[word3]] \
        = (trigrams_dostoevsky[ngram]/ bigrams_dostoevsky[word1 + ' ' + word2])

In [109]:
bi_matrix_tolstoy = np.zeros((len(unigrams_tolstoy), 
                   len(unigrams_tolstoy)))
id2word_tolstoy = list(unigrams_tolstoy)
word2id_tolstoy = {word:i for i, word in enumerate(id2word_tolstoy)}


for ngram in bigrams_tolstoy:
    word1, word2 = ngram.split()
    bi_matrix_tolstoy[word2id_tolstoy[word1]][word2id_tolstoy[word2]] =  (bigrams_tolstoy[ngram]/
                                                                     unigrams_tolstoy[word1])

In [110]:
tri_matrix_tolstoy = np.zeros((len(trigrams_tolstoy), 
                   len(trigrams_tolstoy)))
bi_id2word_tolstoy = list(bigrams_tolstoy)
bi_word2id_tolstoy = {word:i for i, word in enumerate(bi_id2word_tolstoy)}


for ngram in trigrams_tolstoy:
    word1, word2, word3 = ngram.split()
    tri_matrix_tolstoy[bi_word2id_tolstoy[word1 + ' ' + word2]][word2id_tolstoy[word3]] = \
    (trigrams_tolstoy[ngram]/bigrams_tolstoy[word1 + ' ' + word2])

In [111]:
def generate(matrix, id2word, word2id, n=100, start='<start>'):
    text = []
    current_idx = word2id[start]
    
    for i in range(n):
        
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        text.append(id2word[chosen])
        
        if id2word[chosen] == '<end>':
            chosen = word2id['<start>']
        current_idx = chosen
    
    return ' '.join(text)

In [130]:
def tri_generate(matrix, id2word, uni_id2word, word2id, n=100):
    text = []
    for i in range(n):
        chosen = np.random.choice(matrix.shape[1],
                                p=matrix[word2id['<start> <start>']])       
        text.append(id2word[chosen] + " " + uni_id2word[chosen])
        if '<end>' in id2word[chosen]:
                    chosen = np.random.choice(matrix.shape[1],
                                p=matrix[word2id['<start> <start>']])

    return ' '.join(text)

In [131]:
print(tri_generate(tri_matrix_dostoevsky, bi_id2word_dostoevsky, id2word_dostoevsky, bi_word2id_dostoevsky).replace
      ('<end>', '\n').replace('<start>', ' '))


выдают 
 и глава viii пусть двадцать лет сцена выдают 
 и минуту как говорили выдают 
 и выдают 
 и сдаче губернии вдруг не раздавил какова сколько их домового уме и я http://royallib.ru все все по деревням глава двадцать лет сцена   спасибо спасибо по деревням глава бес нас по в городе евангелие положение «гонимого» привычка даже и прекратил выдают 
 и стадо свиней он случившееся побежали видевшие он например но да рядом юноша уме и я не раздавил какова тем и мнение можно так одним степана трофимовича скажу чтения 
 хоть   домового тут минуту как говорили не отличавшемся — сдаче губернии вдруг стадо свиней он лет к это   домового тут минуту как говорили привычка привела уверь глава viii пусть убей следа не бес нас по он например но книгу в в одно время диссертация стадо свиней он между тем вообще книгу в в   домового тут стадо свиней он книгу в в глава viii пусть свиней и бесы и что очень стадо свиней он глава viii пусть стадо свиней он но уже успел бес нас по сдаче губернии вдруг кни

In [132]:
print(generate(bi_matrix_dostoevsky, id2word_dostoevsky, word2id_dostoevsky).
      replace('<end>', '\n').replace('<start> <start>', ' ').replace('<start>', ' '))

не далее как принято было поспешно запрещено и одно время в ежемесячном и ганзеатическом значении немецкого городка ганау в ежемесячном и столь странных событий происшедших в совершенной невинности ему между 1413 и даже науки хотя впрочем в озеро и рассказали им 
     какова же начинает новую жизнь с оттенком высшего юмора 
   за напечатанную первую половину 
 пусть эти хоры поют о совершенной невинности ему вовсе не видно да рядом случается 
     и даровитейший человек в прошлом году предлагал степану трофимовичу ее напечатать здесь — его а если разговаривают то обладатель


In [133]:
print(tri_generate(tri_matrix_tolstoy, bi_id2word_tolstoy, id2word_tolstoy, bi_word2id_tolstoy).replace
      ('<end>', '\n').replace('<start>', ' '))

и кучер да неожиданно уличены вслед – да «ах над ним и из театра ужасно каренина» – – – это это – да «ах бегали по он каренина» – – случается с матвей – да «ах черная кухарка глаза каренина» – – но там она сморщил лоб «всему каренина» – – бегали по он что это «там кухарка и «да тут он увидав каренина» – – к себе пускай взгляд на лев каждая несчастливая может виной эта наверху каренина» – – каренина» – – кухарка и «да взгляд на лев все счастливые с каренина» – – вчера со но что это «там долли неподвижно ответ но там она каренина» – – в этом-то ясно виной эта наверху театра веселый ай но там она удивлению не нехорошо каренина» – – ко дню вместо но там она несчастлива по-своему положение каренина» – – каренина» – – каренина» – – продолжалось уже дети кухарка и «да взгляд на лев каренина» – – продолжалось уже дети но там она а в много – да «ах света пробившуюся эту каренина» – – каренина» – – каждая несчастливая может бегали по он каренина» – – каренина» – – каренина» – –   л.н л.н каренин

In [134]:
print(generate(bi_matrix_tolstoy, id2word_tolstoy, word2id_tolstoy).
      replace('<end>', '\n').replace('<start> <start>', ' ').replace('<start> ', ' '))

 – подумал степан аркадьич помолчал 
 степан аркадьич не виноват 
 что виной эта вечно озабоченная и себя впечатления из этой ссоры князь степан аркадьич помолчал 
  – образуется 
   он не позволял себе 
     «попробовать хотите» – слава богу – сказал он сделал 
   лев николаевич толстой часть пятая часть четвертая часть вторая часть четвертая часть седьмая часть восьмая librs.net 
 он вспомнил вдруг вскочил сел на диван и домочадцы чувствовали что случается с припасами для жены если б ожидал что виной эта глупая улыбка» – все смешалось в


In [135]:

# Задание 2. (5 баллов) 
# При помощи gensim.models.Phrases реализуйте byte-pair-encoding,
# про который говорилось на первом семинаре 
# (https://github.com/mannefedov/compling_nlp_hse_course/blob/master/notebooks/Preprocessing.ipynb) 
# А именно 1) возьмите любой текст; разбейте его на предложения, а каждое предложение разбейте на 
# отдельные символы (не потеряйте пробелы) 2) обучите gensim.models.Phrases на полученных
# символьных предложениях 3) примените полученный нграммер к этим символьным предложениям 
# 4) повторите 2 и 3 N количество раз, чтобы начали получаться целые слова
# Параметры в gensim.models.Phrases влияют на количество получаемых нграммов после каждого 
# прохода, поэтому не забудьте их настроить

In [136]:

from gensim.summarization.textcleaner import split_sentences
from gensim.models.phrases import Phrases


In [137]:
sent_tokens  = sent_tokenize(dostoevsky, 'russian')[:10]
symbol_tokens = [list(i) for i in sent_tokens]
symbol_tokens[0][:10]

['С', 'п', 'а', 'с', 'и', 'б', 'о', ',', ' ', 'ч']

In [138]:
p = Phrases(symbol_tokens, scoring='npmi', min_count=1, threshold=0)

In [139]:
p1 = Phrases(p[symbol_tokens], scoring='npmi', min_count=1, threshold=0.1 )
p2 = Phrases(p1[p[symbol_tokens]], scoring='npmi', min_count=1, threshold=-1 )
p3 = Phrases(p2[p1[p[symbol_tokens]]], scoring='npmi', min_count=1, threshold=-1 )
p4 = Phrases(p3[p2[p1[p[symbol_tokens]]]], scoring='npmi', min_count=1, threshold=1 )
p5 = Phrases(p4[p3[p2[p1[p[symbol_tokens]]]]], scoring='npmi', min_count=1, threshold=1 )

In [140]:

print(list(p[symbol_tokens][:1]), '\n\n\n')
print(list(p1[p[symbol_tokens]][:1]), '\n\n\n')
print(list(p3[p2[p1[p[symbol_tokens]]]][:1]), '\n\n\n')
print(list(p3[p2[p1[p[symbol_tokens]]]][:1]), '\n\n\n' )
print(list(p4[p3[p2[p1[p[symbol_tokens]]]]][:1]),'\n\n\n')


[['С_п', 'а_с', 'и_б', 'о_,', ' _ч', 'т_о', ' _с', 'к_а', 'ч_а', 'л_и', ' _к', 'н_и', 'г_у', ' _в', ' _б', 'е_с', 'п_л', 'а_т', 'н_о', 'й_ ', 'э_л', 'е_к', 'т_р', 'о_н', 'н_о', 'й_ ', 'б_и', 'б_л', 'и', 'о_т', 'е_к', 'е_ ', 'R_o', 'y_a', 'l_l', 'i_b', '._r', 'u_:', ' _h', 't_t', 'p_:', '/_/', 'r_o', 'y_a', 'l_l', 'i_b', '._r', 'u_\n', '\n_В', 'с', 'е_ ', 'к_н', 'и_г', 'и_ ', 'а_в', 'т_о', 'р_а', ':_ ', 'h_t', 't_p', ':_/', '/_r', 'o_y', 'a_l', 'l_i', 'b_.', 'r_u', '/_a', 'u_t', 'h_o', 'r_/', 'd_o', 's_t', 'o_e', 'v_s', 'k_i', 'y__', 'f_e', 'd_o', 'r_.', 'h_t', 'm_l', '\n_\n', 'Э_т', 'а_ ', 'ж_е', ' _к', 'н_и', 'г_а', ' _в', ' _д', 'р_у', 'г_и', 'х_ ', 'ф_о', 'р_м', 'а_т', 'а_х', ':_ ', 'h_t', 't_p', ':_/', '/_r', 'o_y', 'a_l', 'l_i', 'b_.', 'r_u', '/_b', 'o_o', 'k_/', 'd_o', 's_t', 'o_e', 'v_s', 'k_i', 'y__', 'f_e', 'd_o', 'r_/', 'b_e', 's_i', '._h', 't_m', 'l_\n', '\n_П', 'р_и', 'я_т', 'н_о', 'г_о', ' _ч', 'т_е', 'н_и', 'я_!']] 



[['С_п_а_с', 'и_б_о_,', ' _ч_т_о', ' _с_к_а', 'ч_а_л_